<a href="https://colab.research.google.com/github/jordanmsouza/TechChallenge_Fase3_Grupo4/blob/develop/TechChallenge_Fase3_Grupo4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Montando o Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Instalação de dependências:

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install triton
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets


### Importando Bibliotecas

In [ ]:
import json
import pandas as pd
from datasets import load_dataset
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from transformers import pipeline

### Configurações do modelo para fine-tuning

In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

### Modelos Compativeis

In [ ]:
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]

### Formatando o Dataset de produtos

In [ ]:
def format_dataset_into_model_input(data):
    # Inicializando as listas para armazenar os dados
    instructions = []
    inputs = []
    outputs = []

    # Alterando a formatação para título e descrição
    for example in data:
        title = example['title']
        content = example['content']
        instruction = "Descreva o produto de forma estilizada."

        instructions.append(instruction)
        inputs.append(title)
        outputs.append(content)

    # Criando o dicionário final
    formatted_data = {
        "instruction": instructions,
        "input": inputs,
        "output": outputs
    }

    # Salvando o resultado em um arquivo JSON (caso necessário)
    formatted_json_path = "/content/drive/MyDrive/trn.json/formatted_amazon_product_data.json"
    with open(formatted_json_path, 'w') as output_file:
        json.dump(formatted_data, output_file, indent=4)

    print(f"Dataset formatado salvo em: {formatted_json_path}")

    return formatted_json_path

### Declarando Constantes e Formatando o Dataset da Amazon

In [ ]:
# Caminho do arquivo JSON original
DATA_PATH = '/content/drive/MyDrive/trn.json/trn.json'

# Carregar o dataset do arquivo JSON
dataset = load_dataset("json", data_files=DATA_PATH)

# Caminho do arquivo CSV que será gerado
OUTPUT_PATH_DATASET = format_dataset_into_model_input(dataset['train'] if 'train' in dataset else dataset)


### Carregando o modelo pré treinado

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-70B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

### Preparando a inferência do modelo ainda não ajustado com Fine Tuning

In [ ]:
FastLanguageModel.for_inference(model)

### Função para testar o modelo

In [ ]:
def generate_description(title):
    input_text = f"Descreva o produto de forma estilizada.\nTítulo: {title}\nDescrição:"
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=128)
    description = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return description

### Testando o modelo sem fine tuning

In [ ]:
title = "Smartphone Android 64GB"
description = generate_description(title)
print(f"Título: {title}\nDescrição: {description}")

### Ajustes finos do LoRA

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

### Prompt para a descrição dos produtos

In [ ]:
alpaca_prompt = """Abaixo está uma instrução que descreve a tarefa, juntamente com um título que fornece mais contexto. Escreva uma resposta que conclua a descrição de forma apropriada.

### Instrução:
{}

### Título:
{}

### Descrição:
{}"""

EOS_TOKEN = tokenizer.eos_token

### Função para formatar o Prompt

In [ ]:
def formatting_prompts_func(data):
    instructions = data["instruction"]
    inputs = dataset["input"]
    outputs = data["output"]
    texts = []

    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}

### Carregar o Dataset

In [ ]:
dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET)
formatted_dataset = dataset['train'].map(formatting_prompts_func, batched=True)

In [ ]:
formatted_dataset

In [ ]:
dataset['train'][0]

### Configurações do Trainer

In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,  # Usando o dataset formatado e o split 'train'
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

### Treinando o Modelo

In [ ]:
trainer_stats = trainer.train()

### Salvando o Modelo

In [ ]:
# Salvar o modelo ajustado
model.save_pretrained("/content/drive/MyDrive/trn.json/lora_model")
tokenizer.save_pretrained("/content/drive/MyDrive/trn.json/lora_model")

### Reutilizando o modelo já treinado

In [ ]:
# Caminho para o modelo salvo
model_path = "/content/drive/MyDrive/trn.json/lora_model"

# Carregar o modelo e o tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(model_path)

# Preparar o modelo para inferência
FastLanguageModel.for_inference(model)

### Testando o modelo com fine tuning

In [ ]:
title = "Smartphone Android 64GB"
description = generate_description(title)
print(f"Título: {title}\nDescrição: {description}")